In [4]:
!pip install pytest-mock

In [5]:
!pip install pytest pytest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4


In [6]:
%%writefile order_module.py
import asyncio
from datetime import datetime
from enum import Enum

# Mock classes

class OrderRequestType(Enum):
    SUBMIT = 'submit'
    UPDATE = 'update'
    CANCEL = 'cancel'



class OrderRequestStatus(Enum):
    UNPROCESSED = 'unprocessed'
    ERROR = 'error'


class OrderResponse:
    UNPROCESSED = 'unprocessed'

    def __init__(self, is_error=False):
        self.is_error = is_error

# Main OrderRequest Class .

class OrderRequest:
    def __init__(self, time: datetime, order_id: int, tag: str):
        self.time = time
        self.order_id = order_id
        self.tag = tag
        self.response = OrderResponse()
        self.status = OrderRequestStatus.UNPROCESSED


    async def set_response(self, response: OrderResponse, status: OrderRequestStatus = OrderRequestStatus.ERROR):
        if response is None:
            raise ValueError("Response cannot be None")

        await asyncio.sleep(1)
        self.status = OrderRequestStatus.ERROR if response.is_error else status
        self.response = response

    def __str__(self):
        return f"OrderRequest(time={self.time}, order_id={self.order_id}, tag={self.tag}, status={self.status}, response={self.response})"







Overwriting order_module.py


In [7]:
%%writefile test_order_module.py
import asyncio
import pytest
from datetime import datetime
from unittest.mock import AsyncMock
from order_module import OrderRequest, OrderResponse, OrderRequestStatus,AsyncEventLoop


@pytest.mark.asyncio
async def test_order_request_set_response_success(mocker):
    order_request = OrderRequest(datetime.utcnow(), 0, "Test Order")

    mock_response = AsyncMock(spec=OrderResponse)
    mock_response.is_error = False

    await order_request.set_response(mock_response, OrderRequestStatus.UNPROCESSED)
    assert order_request.status == OrderRequestStatus.UNPROCESSED
    assert order_request.response.is_error is False


@pytest.mark.asyncio
async def test_order_request_set_response_error(mocker):
    order_request = OrderRequest(datetime.utcnow(), 1, "Test Order")

    mock_response = AsyncMock(spec=OrderResponse)
    mock_response.is_error = True

    await order_request.set_response(mock_response, OrderRequestStatus.UNPROCESSED)

    assert order_request.status == OrderRequestStatus.ERROR
    assert order_request.response.is_error is True





Overwriting test_order_module.py


In [8]:
!pytest test_order_module.py --asyncio-mode=auto

/usr/local/lib/python3.10/dist-packages/pytest_asyncio/plugin.py:208: PytestDeprecationWarning: The configuration option "asyncio_default_fixture_loop_scope" is unset.
The event loop scope for asynchronous fixtures will default to the fixture caching scope. Future versions of pytest-asyncio will default the loop scope for asynchronous fixtures to function scope. Set the default fixture loop scope explicitly in order to avoid unexpected behavior in the future. Valid fixture loop scopes are: "function", "class", "module", "package", "session"

  warnings.warn(PytestDeprecationWarning(_DEFAULT_FIXTURE_LOOP_SCOPE_UNSET))
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0
rootdir: /content
plugins: asyncio-0.24.0, mock-3.14.0, anyio-3.7.1, typeguard-4.3.0
asyncio: mode=auto, default_loop_scope=None
collected 2 items                                                                   